In [ ]:
!pip install transformers
!pip install datasets
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
import torchaudio
import re
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from datasets import load_metric, Dataset
from ast import literal_eval
from functools import lru_cache
import random
import IPython.display as ipd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
language_code = 'ar'
language_name = 'arabic'
base_model = "facebook/wav2vec2-large-xlsr-53"

data_dir = "./drive/My Drive/Senior Project/Resources/data"
bucket_name = "hearsome-sagemaker-datasets"

output_models_dir = f"./workspace/output_models/{language_code}/wav2vec2-large-xlsr-{language_name}-demo"
new_output_models_dir = f"./drive/My Drive/Senior Project/Resources/workspace/output_models/{language_code}/wav2vec2-large-xlsr-{language_name}"

In [ ]:
test = pd.read_csv(f"{data_dir}/test.csv")
test.head()

,filename,start_time,end_time,sentence
0,A0003_S002_0_G0001,747.799,748.478,يعني
1,A0003_S002_0_G0001,536.509,538.449,والكلام هذا كله، كل دوت
2,A0003_S002_0_G0001,640.429,642.099,بين الطالب والمدرس
3,A0003_S002_0_G0001,927.440,929.378,ممكن الطالب آه
4,A0003_S002_0_G0001,467.600,469.119,ايه، جميل جداً


In [ ]:
test_dataset = Dataset.from_pandas(test)

In [ ]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'
resampler = torchaudio.transforms.Resample(48_000, 16_000)

@lru_cache(maxsize=None)
def load_wav(path_to_file):
    try:
        return torchaudio.load(path_to_file)
    except RuntimeError as rtm_err:
        return None

def speech_file_to_array_fn(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '',
                               batch["sentence"]).lower()

    start_time, end_time = batch["start_time"], batch["end_time"]
    
    path_to_file = data_dir + "/wav/" + batch["filename"] + ".wav"
    tmp = load_wav(path_to_file)    
    
    speech_array, sampling_rate = tmp

    start_sample = int(float(start_time)*sampling_rate)
    end_sample = int(float(end_time)*sampling_rate)
    size = list(speech_array[0].shape)[0]
    
    batch["speech"] = speech_array[0][start_sample:end_sample].squeeze().numpy()

    
    return batch

In [ ]:
test_dataset = test_dataset.map(speech_file_to_array_fn)

Parameter 'function'=<function speech_file_to_array_fn at 0x7f78b03f3f80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1311 [00:00<?, ?ex/s]

In [ ]:
rand_int = random.randint(0, 1000)
ipd.Audio(data=np.asarray(test_dataset["speech"][rand_int]),
          autoplay=True, rate=16000)

In [ ]:
wer_metric = load_metric("wer")
processor = Wav2Vec2Processor.from_pretrained(new_output_models_dir)
model = Wav2Vec2ForCTC.from_pretrained(new_output_models_dir)
model.to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elemen

In [ ]:
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000,
                       return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch

In [ ]:
result = test_dataset.map(evaluate, batched=True, batch_size=8)

  0%|          | 0/164 [00:00<?, ?ba/s]

In [ ]:
print("WER: {:2f}".format(100 *
                          wer_metric.compute(predictions=result["pred_strings"],
                                            references=result["sentence"])))

WER: 29.375516


In [ ]:
for row in result:
  print(row["pred_strings"] + " || " + row["sentence"])

يعني || يعني
والكلام ده كله كل د وت || والكلام هذا كله، كل دوت
بين الطالب والمدرس || بين الطالب والمدرس
ممكن الطالب اه || ممكن الطالب آه
اه جميل جدا || ايه، جميل جداً
وخلت التعليم || و خلت التعليم
بحيث انها توصل له المعلومة || بحيث انها توصله المعلومة
اه ويستطيع انه هو يتكلم في ثقافات عديدة || اي و يستطيع أنه هو يتكلم في ثقافات عديدة
كثير من الناس بتتجاهل النقطة دي ما بتتكلمشي فيها || كثير من الناس بتتجاهل النقطة هذه، وما بتتكلمش فيها
اه جميل جدا السؤال دهوك || آه، جميل جداَ السؤال هذا، و آه
حتى لو المدرس اه الله انتجوبته وبرافه عليكو الكلام ده كله || حتى لو المدرس آه قال له إن جاوبت و برافه عليك و الكلام دا كله
اه ما يخلش في مشاكل ويخلي الانسان عنده ثقةب نفسه || آه ما يخليش فيه مشاكل ، ويخلي الإنسان عنده ثقة بنفسه
يتعلم اه || يتعلم آه
اكيد ب تطور الح المهارات الحياتية || اكيد بي آه تطور الحا المهارات الحياتية
اه يسال الطالب سؤال والطالب يجوب عليه || آه يسأل الطالب سؤال و الطالب يجاوب عليه
والوهيت || والفيديوهات
من قبل زمالائه انه هم يشجعوا || من قبل زملائه إن هم يشجعوا
الاون لاين سهل 